In [1]:
import random
import torch
try : 
    torch.multiprocessing.set_start_method('spawn')
except : 
    pass
import numpy as np

class ReplayBuffer:
    def __init__(self, capacity, device):
        self.capacity = capacity # capacity of the buffer
        self.data = []
        self.index = 0 # index of the next cell to be filled
        self.device = device
    def append(self, s, a, r, s_, d):
        if len(self.data) < self.capacity:
            self.data.append(None)
        self.data[self.index] = (s, a, r, s_, d)
        self.index = (self.index + 1) % self.capacity
    def sample(self, batch_size):
        batch = random.sample(self.data, batch_size)
        return list(map(lambda x:torch.Tensor(np.array(x)).to(self.device), list(zip(*batch))))
    def __len__(self):
        return len(self.data)
import torch

def greedy_action(network, state):
    device = "cuda" if next(network.parameters()).is_cuda else "cpu"
    state= np.log(state+1e-9)
    with torch.no_grad():
        Q = network(torch.Tensor(state).to(device))
        return torch.argmax(Q, dim=1).cpu()
    
import numpy as np
import torch
import torch.nn as nn
from copy import deepcopy

class dqn_agent:
    def __init__(self, config, model,pi =None):
        self.pi = pi
        device = "cuda" if next(model.parameters()).is_cuda else "cpu"
        self.nb_actions = config['nb_actions']
        self.gamma = config['gamma'] if 'gamma' in config.keys() else 0.95
        self.batch_size = config['batch_size'] if 'batch_size' in config.keys() else 100
        buffer_size = config['buffer_size'] if 'buffer_size' in config.keys() else int(1e5)
        self.memory = ReplayBuffer(buffer_size,device)
        self.epsilon_max = config['epsilon_max'] if 'epsilon_max' in config.keys() else 1.
        self.epsilon_min = config['epsilon_min'] if 'epsilon_min' in config.keys() else 0.01
        self.epsilon_stop = config['epsilon_decay_period'] if 'epsilon_decay_period' in config.keys() else 1000
        self.epsilon_delay = config['epsilon_delay_decay'] if 'epsilon_delay_decay' in config.keys() else 20
        self.epsilon_step = (self.epsilon_max-self.epsilon_min)/self.epsilon_stop
        self.model = model 
        self.target_model = deepcopy(self.model).to(device)
        self.criterion = config['criterion'] if 'criterion' in config.keys() else torch.nn.MSELoss()
        lr = config['learning_rate'] if 'learning_rate' in config.keys() else 0.001
        self.optimizer = config['optimizer'] if 'optimizer' in config.keys() else torch.optim.Adam(self.model.parameters(), lr=lr)
        self.nb_gradient_steps = config['gradient_steps'] if 'gradient_steps' in config.keys() else 1
        self.update_target_strategy = config['update_target_strategy'] if 'update_target_strategy' in config.keys() else 'replace'
        self.update_target_freq = config['update_target_freq'] if 'update_target_freq' in config.keys() else 20
        self.update_target_tau = config['update_target_tau'] if 'update_target_tau' in config.keys() else 0.005
        self.monitoring_nb_trials = config['monitoring_nb_trials'] if 'monitoring_nb_trials' in config.keys() else 0

    def MC_eval(self, env, nb_trials):   # NEW NEW NEW
        MC_total_reward = []
        MC_discounted_reward = []
        for _ in range(nb_trials):
            x,_ = env.reset()
            done = False
            trunc = False
            total_reward = 0
            discounted_reward = 0
            step = 0
            while not (done or trunc):
                a = greedy_action(self.model, x)
                y,r,done,trunc,_ = env.step(a)
                x = y
                total_reward += r
                discounted_reward += self.gamma**step * r
                step += 1
            MC_total_reward.append(total_reward)
            MC_discounted_reward.append(discounted_reward)
        return np.mean(MC_discounted_reward), np.mean(MC_total_reward)
    
    def V_initial_state(self, env, nb_trials):   # NEW NEW NEW
        with torch.no_grad():
            for _ in range(nb_trials):
                val = []
                x,_ = env.reset()
                val.append(self.model(torch.Tensor(x).unsqueeze(0).to(device)).max().item())
        return np.mean(val)
    
    def gradient_step(self):
        if len(self.memory) > self.batch_size:
            X, A, R, Y, D = self.memory.sample(self.batch_size)
            X,Y = (X+1e-9).log() , (Y+1e-9).log()
            QYmax = self.target_model(Y).max(1)[0].detach()
            update = torch.addcmul(R, 1-D, QYmax, value=self.gamma)
            QXA = self.model(X).gather(1, A.to(torch.long).unsqueeze(1))
            loss = self.criterion(QXA, update.unsqueeze(1))
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(),0.5)
            self.optimizer.step() 
    
    def train(self, env, max_episode):
        episode_return = []
        MC_avg_total_reward = []   # NEW NEW NEW
        MC_avg_discounted_reward = []   # NEW NEW NEW
        V_init_state = []   # NEW NEW NEW
        episode = 0
        episode_cum_reward = 0
        state, _ = env.reset()
        epsilon = self.epsilon_max
        step = 0
        while episode < max_episode:
            # update epsilon
            if step > self.epsilon_delay:
                epsilon = max(self.epsilon_min, epsilon-self.epsilon_step)
            # select epsilon-greedy action
            if episode <1 and self.pi is not None:
                action = self.pi.get_action(state)
            else :
                if np.random.rand() < epsilon:
                    action = env.action_space.sample()
                    
                else:
                    action = greedy_action(self.model, state)
            # step
            next_state, reward, done, trunc, _ = env.step(action)
            for u in range(env.observation_space.shape[0]) :
                self.memory.append(state[u], action[u], reward[u], next_state[u], done[u])
            episode_cum_reward += reward
            # train
            for _ in range(self.nb_gradient_steps): 
                self.gradient_step()
            # update target network if needed
            if self.update_target_strategy == 'replace':
                if step % self.update_target_freq == 0: 
                    self.target_model.load_state_dict(self.model.state_dict())
            if self.update_target_strategy == 'ema':
                target_state_dict = self.target_model.state_dict()
                model_state_dict = self.model.state_dict()
                tau = self.update_target_tau
                for key in model_state_dict:
                    target_state_dict[key] = tau*model_state_dict[key] + (1-tau)*target_state_dict[key]
                self.target_model.load_state_dict(target_state_dict)
            # next transition
            step += 1
            if any(done) or any(trunc):
                episode += 1
                # Monitoring
                if self.monitoring_nb_trials>0:
                    MC_dr, MC_tr = self.MC_eval(env, self.monitoring_nb_trials)    # NEW NEW NEW
                    V0 = self.V_initial_state(env, self.monitoring_nb_trials)   # NEW NEW NEW
                    MC_avg_total_reward.append(MC_tr)   # NEW NEW NEW
                    MC_avg_discounted_reward.append(MC_dr)   # NEW NEW NEW
                    V_init_state.append(V0)   # NEW NEW NEW
                    episode_return.append(episode_cum_reward)   # NEW NEW NEW
                    print("Episode ", '{:2d}'.format(episode), 
                          ", epsilon ", '{:6.2f}'.format(epsilon), 
                          ", batch size ", '{:4d}'.format(len(self.memory)), 
                          ", ep return ", '{:e}'.format(episode_cum_reward), 
                          ", MC tot ", '{:6.2f}'.format(MC_tr),
                          ", MC disc ", '{:6.2f}'.format(MC_dr),
                          ", V0 ", '{:6.2f}'.format(V0),
                          sep='')
                else:
                    episode_return.append(episode_cum_reward)
                    print("Episode ", '{:2d}'.format(episode), 
                          ", epsilon ", '{:6.2f}'.format(epsilon), 
                          ", batch size ", '{:4d}'.format(len(self.memory)), 
                          ", ep return ", '{:e}'.format(descale(episode_cum_reward.mean())), 
                          sep='')

                
                state, _ = env.reset()
                episode_cum_reward = 0
            else:
                state = next_state
            if descale(np.mean(episode_cum_reward))>4e10 : 
                return episode_return, MC_avg_discounted_reward, MC_avg_total_reward, V_init_state
        return episode_return, MC_avg_discounted_reward, MC_avg_total_reward, V_init_state

In [2]:
import gymnasium as gym
import env_hiv
from typing import Protocol
import numpy as np
from tqdm import tqdm
import pickle
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Declare network
from env_hiv import HIVPatient
env = HIVPatient(domain_randomization=True)
from gymnasium.wrappers import TimeLimit
from gymnasium.wrappers import TransformReward
def rew(state,action, env) : 
    return-(
                env.Q * state[4]
                + env.R1 * action[0] ** 2
                + env.R2 * action[1] ** 2
                - env.S * state[5]
            )
low_r, high_r = rew(env.lower, [1,1],env),rew(env.upper, [1,1],env)
env = TransformReward(env, lambda r: (r-low_r)/(high_r-low_r))
env = TimeLimit(env,200)
env =  gym.vector.AsyncVectorEnv([lambda : env for i in range(10)])
def descale(r) :
    return r * (high_r - low_r) +low_r

state_dim = env.observation_space.shape[1]
n_action = env.action_space.nvec[0]
nb_neurons=256
DQN = torch.nn.Sequential(nn.Linear(state_dim, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(), 
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(), 
                          nn.Linear(nb_neurons, n_action)).to(device)


config = {'nb_actions': n_action,
          'learning_rate': 1e-2,
          'gamma': 0.9,
          'buffer_size': 1000000,
          'epsilon_min': 0.01,
          'epsilon_max': 0.01,
          'epsilon_decay_period': 1000,
          'epsilon_delay_decay': 20,
          'batch_size': 1024,
          'gradient_steps':3,
          'update_target_strategy': 'replace', # or 'ema'
          'update_target_freq': 50,
          'update_target_tau': 0.005,
          'criterion': torch.nn.SmoothL1Loss(),
          'monitoring_nb_trials': 0}

from train_cma import ProjectAgent
with open('DQNAGENTS/saved3.pkl', 'rb') as f:  # open a text file
    dqn = pickle.load( f)['dqn'] # serialize the list
agent = dqn_agent(config, DQN)
agent.model = deepcopy(dqn.to(device))
agent.target_model = deepcopy(dqn.to(device))
agent.train(env, 500)


Episode  1, epsilon   0.01, batch size 2000, ep return 3.041952e+10
Episode  2, epsilon   0.01, batch size 4000, ep return 3.423050e+10
Episode  3, epsilon   0.01, batch size 6000, ep return 3.058209e+10
Episode  4, epsilon   0.01, batch size 8000, ep return 2.698586e+10
Episode  5, epsilon   0.01, batch size 10000, ep return 2.657617e+10
Episode  6, epsilon   0.01, batch size 12000, ep return 2.848544e+10
Episode  7, epsilon   0.01, batch size 14000, ep return 2.312518e+10
Episode  8, epsilon   0.01, batch size 16000, ep return 2.974235e+10
Episode  9, epsilon   0.01, batch size 18000, ep return 2.876981e+10
Episode 10, epsilon   0.01, batch size 20000, ep return 2.971053e+10
Episode 11, epsilon   0.01, batch size 22000, ep return 2.518730e+10
Episode 12, epsilon   0.01, batch size 24000, ep return 2.801637e+10
Episode 13, epsilon   0.01, batch size 26000, ep return 3.336907e+10
Episode 14, epsilon   0.01, batch size 28000, ep return 3.086545e+10
Episode 15, epsilon   0.01, batch size

KeyboardInterrupt: 

In [3]:
def greedy_action(network, state):
    device = "cuda" if next(network.parameters()).is_cuda else "cpu"
    state= np.log(state+1e-9)
    if len(torch.Tensor(state).shape)==1 :
        state = torch.Tensor(state).unsqueeze(0)
    with torch.no_grad():
        Q = network(torch.Tensor(state).to(device))
        return torch.argmax(Q, dim=1).cpu()
class ProjectAgent:

    def __init__(self) :
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.env =env_hiv.HIVPatient(domain_randomization=False)
        self.config = config
        self.dqn = None
    def act(self, observation: np.ndarray, use_random: bool = False) -> int:
        return greedy_action(self.dqn, observation)

    def save(self, path=""):
        serialized= {"dqn":self.dqn.cpu(), "config":self.config}
        with open('DQNAGENTS/saved3.pkl', 'wb') as f:  # open a text file
            pickle.dump(serialized, f) # serialize the list
    def load(self):
        with open('DQNAGENTS/saved3.pkl', 'rb') as f:  # open a text file
            saved = pickle.load( f) # serialize the list
        self.dqn= saved["dqn"].to(self.device)
        try : 
            x,_ = self.env.reset()
            self.act(x)
        except : 
            raise Exception("Actor incompatible with environnement")
    
    def train(self):
        return self.agent.train(self.env,self.config['epochs'])
Pagent = ProjectAgent()
Pagent.dqn = agent.model

from evaluate import evaluate_HIV, evaluate_HIV_population

In [4]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(seed)

# Keep the following lines to evaluate your agent unchanged.
score_agent: float = evaluate_HIV(agent=Pagent, nb_episode=1)
score_agent_dr: float = evaluate_HIV_population(agent=Pagent, nb_episode=15)
with open(file="score.txt", mode="w") as f:
    f.write(f"{score_agent}\n{score_agent_dr}")
print("{:e}".format(score_agent),"{:e}".format(score_agent_dr))

3.516853e+10 3.248472e+10


In [12]:
import os
names = os.listdir("DQNAGENTS/")
models= []
for name in names :
    with open("DQNAGENTS/"+name, 'rb') as f:  # open a text file
        models.append(pickle.load( f)['dqn']) # serialize the list


In [87]:

class ProjectAgent:
    def __init__(self) :
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.env =env_hiv.HIVPatient(domain_randomization=False)
        self.config = config
        self.models = []
    def act(self, observation: np.ndarray, use_random: bool = False) -> int:
        observation = (torch.Tensor(observation).to(torch.float32)+1e-9).log()
        logits = [m(observation) for m in self.models[:3]]
        #logits =[models[0](observation)]+[models[2](observation)]
        logits =torch.softmax(torch.stack(logits,0),1).mean(0)
        return torch.argmax(logits).item()

    def save(self, path=""):
        serialized= {"models" :self.models}
        with open(path+'models.pkl', 'wb') as f:  # open a text file
            pickle.dump(serialized, f) # serialize the list
    def load(self):
        with open('models.pkl', 'rb') as f:  # open a text file
            saved = pickle.load( f) # serialize the list
        self.models= saved["models"]
        try : 
            x,_ = self.env.reset()
            self.act(x)
        except : 
            raise Exception("Actor incompatible with environnement")
agent = ProjectAgent()
agent.models= models
agent.save()
agent.load()

In [88]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(seed)

# Keep the following lines to evaluate your agent unchanged.
score_agent: float = evaluate_HIV(agent=agent, nb_episode=1)
score_agent_dr: float = evaluate_HIV_population(agent=agent, nb_episode=15)
with open(file="score.txt", mode="w") as f:
    f.write(f"{score_agent}\n{score_agent_dr}")
print("{:e}".format(score_agent),"{:e}".format(score_agent_dr))

3.877342e+10 2.528770e+10


In [72]:
names

['saved2.pkl', 'saved0.pkl', 'saved3.pkl', 'saved1.pkl']